In [2]:
import os

import pandas as pd
import Python as py

-1.0742968489855099e+17


In [3]:
root = os.getcwd()
df_scheduled = pd.read_csv(root + "/data/scheduled_loan_repayments.csv")
df_actual = pd.read_csv(root + "/data/actual_loan_repayments.csv")
df_balances = py.calculate_df_balances(df_scheduled, df_actual)

In [10]:
def question_1(df_balances):
    # 1. Flag monthly short payments as "missed"
    df_balances["missed"] = df_balances["ActualRepayment"] < df_balances["ScheduledRepayment"]
    
    # 2. A loan defaults if ANY payment is missed
    type1_default = df_balances.groupby("LoanID")["missed"].any()
    
    # 3. Compute percentage type 1 default rate
    default_rate_percent = type1_default.mean() * 100
    
    return default_rate_percent
print(question_1(df_balances))

15.0


In [6]:
def question_2(df_scheduled, df_balances):
    
    # 1. Calculate the expected yearly payments (12 months)
    scheduled_yearly = (
        df_scheduled.set_index("LoanID")["ScheduledRepayment"] * 12
    )

    # 2. Actual total payments made (sum of monthly actuals per loan ID - use the groupby function)
    actual_yearly = (
        df_balances.groupby("LoanID")["ActualRepayment"].sum()
    )

    # 3. How much was unpaid
    unpaid_amount = scheduled_yearly - actual_yearly

    # 4. Type 2 default rule: unpaid > 15% of expected
    type2_defaults = unpaid_amount > (0.15 * scheduled_yearly)

    # 5. Percent of loans defaulted
    default_rate_percent = type2_defaults.mean() * 100


    return default_rate_percent
print(question_2(df_scheduled, df_balances))

1.2


In [7]:
def question_3(df_balances):
    """
    Calculate the anualized portfolio CPR (As a %) from the geometric mean SMM.
    SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)
    SMM_mean is calculated as (∏(1+SMM))^(1/12) - 1
    CPR is calcualted as: 1 - (1- SMM_mean)^12

    Args:
        df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function

    Returns:
        float: The anualized CPR of the loan portfolio as a percent.

    """

    df = df_balances.copy()

    # 1. Compute principal paid each month
    df["PrincipalPaid"] = df["LoanBalanceStart"] - df["LoanBalanceEnd"]

    # 2. Compute unscheduled principal
    df["UnscheduledPrincipal"] = df["PrincipalPaid"] - df["ScheduledRepayment"]
    df["UnscheduledPrincipal"] = df["UnscheduledPrincipal"].clip(lower=0)

    # 3. Compute SMM
    df = df[df["LoanBalanceStart"] > 0]   # avoid divide-by-zero
    df["SMM"] = df["UnscheduledPrincipal"] / df["LoanBalanceStart"]

    # 4. Compute geometric mean SMM
    SMM_series = df["SMM"].dropna()

    if len(SMM_series) == 0:
        return 0.0   # Hence, there will be no prepayments

    SMM_mean = ( (1 + SMM_series).prod() ** (1/12) ) - 1 # Formula given

    # 5. CPR conversion
    CPR = 1 - (1 - SMM_mean)**12
    cpr_percent = CPR * 100

    return cpr_percent

print(question_3(df_balances))

-492365028297151.0


In [8]:
def question_3(df_balances):
    """
    Calculate the anualized portfolio CPR (As a %) from the geometric mean SMM.
    SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)
    SMM_mean is calculated as (∏(1+SMM))^(1/12) - 1
    CPR is calcualted as: 1 - (1- SMM_mean)^12

    Args:
        df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function

    Returns:
        float: The anualized CPR of the loan portfolio as a percent.

    """

    df = df_balances.copy()

    df["PrincipalPaid"] = df["LoanBalanceStart"] - df["LoanBalanceEnd"]

    median_sched_prin = (df["ScheduledRepayment"] - df["InterestPayment"]).median()

    df["UnscheduledPrincipal"] = (df["PrincipalPaid"] - median_sched_prin).clip(lower=0)

    df = df[df["LoanBalanceStart"] > 0]

    df["SMM"] = df["UnscheduledPrincipal"] / df["LoanBalanceStart"]

    SMM_series = df["SMM"].clip(lower=0, upper=1).dropna()

    if len(SMM_series) == 0:
        return 0.0

    SMM_mean = ((1 + SMM_series).prod() ** (1/12)) - 1

    CPR = 1 - (1 - SMM_mean)**12

    return CPR * 100
print(question_3(df_balances))

-3.2875406430804326e+50


In [9]:
def question_4(df_balances):
# --- Compute probability of default using Type 2 definition ---
    # Expected yearly payment (ScheduledRepayment * 12)
    scheduled_yearly = (
        df_balances.groupby("LoanID")["ScheduledRepayment"].first() * 12
    )

    # Actual payments Year 1 (Months 1–12)
    actual_yearly = (
        df_balances[df_balances["Month"] <= 12]
        .groupby("LoanID")["ActualRepayment"]
        .sum()
    )

    unpaid = scheduled_yearly - actual_yearly
    type2_defaults = unpaid > (0.15 * scheduled_yearly)
    probability_of_default = type2_defaults.mean()  # not percent

    # --- Total loan balance entering Year 2 ---
    year2_balances = df_balances[df_balances["Month"] == 13]["LoanBalanceStart"]
    total_loan_balance = year2_balances.sum()

    # --- Loss calculation ---
    recovery_rate = 0.80
    total_loss = probability_of_default * total_loan_balance * (1 - recovery_rate)

    return total_loss
print(question_4(df_balances))

0.0


In [18]:
def question_2(df_scheduled, df_balances):

    # 1. Calcualtion of the Scheduled yearly payments (monthly payment * 12)
    scheduled_yearly = df_scheduled.set_index("LoanID")["ScheduledRepayment"] * 12
    
    # 2. Calcualtion of the Actual yearly payments (monthly payment * 12)
    actual_yearly = df_balances.groupby("LoanID")["ActualRepayment"].sum()
    
    # 3. Calculate the amount that was not paid for the year
    unpaid_amount = scheduled_yearly - actual_yearly

    # 4. Apply Type 2 Loan Default Rule - If more than 15% of expected yearly payments are unpaid it returns TRUE
    # unpaid_amount > 15% * expected_yearly - Generate a TRUE/FALSE boolean
    type2_default = unpaid_amount > (0.15 * scheduled_yearly)

    # 5. Compute the percentage of defaulted loans (Convert the boolean to a percentage)
    default_rate_percent = type2_default.mean() * 100

    return default_rate_percent

print(question_2(df_scheduled, df_balances))
    

1.2


In [26]:
def question_1(df_balances):
    """
    Calculate the percent of loans that defaulted as per the type 1 default definition.

    Args:
        df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function

    Returns:
        float: The percentage of type 1 defaulted loans (ie 50.0 not 0.5)
"""
    # 1. Flag each loan payments that were missed/underpaid i.e. TRUE if monthly payment is missed/underpaid
    df_balances["missed"] = df_balances["ActualRepayment"] < df_balances["ScheduledRepayment"]

    # 2. Group each loan by loan ID to check if any unpaid payments per loan exist
    loan_default_flag = df_balances.groupby("LoanID")["missed"].any() # If True the loan defaulted if False the loan is clean

    # 3. Convert boolean flag into percentage
    default_rate_percent = loan_default_flag.mean() * 100
    
    return default_rate_percent
print(question_1(df_balances))

15.0


In [31]:
def question_3(df_balances):
    """
    Calculate the anualized portfolio CPR (As a %) from the geometric mean SMM.
    SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)
    SMM_mean is calculated as (∏(1+SMM))^(1/12) - 1
    CPR is calcualted as: 1 - (1- SMM_mean)^12

    Args:
        df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function

    Returns:
        float: The anualized CPR of the loan portfolio as a percent.

    """
    # Create alias for df_balances
    df = df_balances.copy()

    # 1. Calculate the total pricipal paid in each month
    df["PrincipalPaid"] = df["LoanBalanceStart"] - df["LoanBalanceEnd"]

    # 2. Calcualte the scheduled principal for each month
    df["ScheduledPrincipal"] = df["ScheduledRepayment"] - df["InterestPayment"]

    # 3. Calculate the unscheduled principal ( Excess principal paid above the scheduled principal)
    df["UnscheduledPrincipal"] = (df["PrincipalPaid"] - df["ScheduledPrincipal"]).clip(lower=0)

    # 4. Filter out months where the loan had no balance
    df = df[df["LoanBalanceStart"] > 0]

    # 5. Calcualte the monthly SMM for each loan 
    df["SMM"] = df["UnscheduledPrincipal"] / df["LoanBalanceStart"] 

    # 6. Calculate the SMM mean over all loan-months
    SMM_series = df["SMM"].dropna() #dropna ensures that only valid SMM values are considered

    if SMM_series.empty:
        return 0.0
        
    SMM_mean = ((1 + SMM_series).prod() ** (1/12)) - 1 #Formual given as (Π(1 + SMM) )^(1/12) – 1

    # 7. Calculate annualised CPR by using SMM_Mean
    CPR = 1 - (1 - SMM_mean) ** 12
    cpr_percent= CPR * 100
    
    return cpr_percent

print(question_3(df_balances))

-1.0742968489855099e+17


In [15]:
def question_4(df_balances):
    """
    Calculate the predicted total loss for the second year in the loan term.
    Use the equation: probability_of_default * total_loan_balance * (1 - recovery_rate).
    The probability_of_default value must be taken from either your question_1 or question_2 answer.
    Decide between the two answers based on which default definition you believe to be the more useful metric.
    Assume a recovery rate of 80%

    Args:
        df_balances (DataFrame): Dataframe created from the 'calculate_df_balances()' function

    Returns:
        float: The predicted total loss for the second year in the loan term.

    """
    
    # 1. Calculate the expected yearly payment (ScheduledRepayment * 12)
    scheduled_yearly = (df_balances.groupby("LoanID")["ScheduledRepayment"].first() * 12)

    #2. Calculate the actual payments for Year 1 per loan id (Months 1–12)
    actual_yearly = (df_balances[df_balances["Month"] <= 12].groupby("LoanID")["ActualRepayment"].sum())

    #3. Calcualte the unpaid loan amount for the year
    unpaid = scheduled_yearly - actual_yearly

    #4. Use type 2 default as the probability of default
    type2_defaults = unpaid > (0.15 * scheduled_yearly) #Apply Type 2 default loan rule
    probability_of_default = type2_defaults.mean()  # not percent

    #5 .Calculate the Total loan balance entering Year 2
    year2_balances = df_balances[df_balances["Month"] == 13]["LoanBalanceStart"]
    total_loan_balance = year2_balances.sum()

    # Loss calculation ---
    recovery_rate = 0.80
    total_loss = probability_of_default * total_loan_balance * (1 - recovery_rate)

    return total_loss

print(question_4(df_balances))

0.0
